# Dask Bags


Dask Bag implements operations like `map`, `filter`, `groupby` and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

Full API documentation is available here: http://docs.dask.org/en/latest/bag-api.html

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 4253 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:4253/status,
Dashboard: http://127.0.0.1:4253/status,Workers: 4
Total threads: 4,Total memory: 15.69 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:4254,Workers: 4
Dashboard: http://127.0.0.1:4253/status,Total threads: 4
Started: Just now,Total memory: 15.69 GiB
Comm: tcp://127.0.0.1:4274,Total threads: 1
Dashboard: http://127.0.0.1:4277/status,Memory: 3.92 GiB
Nanny: tcp://127.0.0.1:4257,


## Create Random Data

We create a random set of record data and store it to disk as many JSON files.  This will serve as our data for this notebook.

In [3]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/0.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/1.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/2.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/3.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/4.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/5.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/6.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/7.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/8.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/9.json']

## Read JSON data

Now that we have some JSON data in a file lets take a look at it with Dask Bag and Python JSON module.

In [5]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [6]:
b.take(2)

({'age': 25,
  'name': ['Cletus', 'Haney'],
  'occupation': 'Radiographer',
  'telephone': '+18300170997',
  'address': {'address': '407 Cielito Pike', 'city': 'Lafayette'},
  'credit-card': {'number': '4738 1562 8032 9005',
   'expiration-date': '07/18'}},
 {'age': 92,
  'name': ['Hank', 'Moran'],
  'occupation': 'Shunter',
  'telephone': '+1-937-909-2177',
  'address': {'address': '341 Sunbeam Boulevard', 'city': 'Enid'},
  'credit-card': {'number': '4640 7218 0241 3616',
   'expiration-date': '02/18'}})

## Map, Filter, Aggregate

We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [7]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30

({'age': 92,
  'name': ['Hank', 'Moran'],
  'occupation': 'Shunter',
  'telephone': '+1-937-909-2177',
  'address': {'address': '341 Sunbeam Boulevard', 'city': 'Enid'},
  'credit-card': {'number': '4640 7218 0241 3616',
   'expiration-date': '02/18'}},
 {'age': 47,
  'name': ['Erline', 'Allison'],
  'occupation': 'Advertising Manager',
  'telephone': '+1-325-806-0524',
  'address': {'address': '1349 Del Sur Hill', 'city': 'Prichard'},
  'credit-card': {'number': '3400 503900 84972', 'expiration-date': '11/23'}})

In [8]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field

('Radiographer', 'Shunter')

In [9]:
b.count().compute()  # Count total number of records

10000

## Chain computations

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [10]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

As with all lazy Dask collections, we need to call `compute` to actually evaluate our result.  The `take` method used in earlier examples is also like `compute` and will also trigger computation.

In [11]:
result.compute()

[('Rig Worker', 15),
 ('Tyre Fitter', 15),
 ('Roadworker', 15),
 ('Garda', 14),
 ('Amusement Arcade Worker', 14),
 ('Sheriff Principal', 14),
 ('Polisher', 14),
 ('Print Finisher', 13),
 ('Industrial Chemist', 13),
 ('Representative', 13)]

## Transform and Store

Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses.  For that we can use methods like `to_textfiles` and `json.dumps`, or we can convert to Dask Dataframes and use their storage systems, which we'll see more of in the next section.

In [12]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.0.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.1.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.2.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.3.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.4.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.5.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.6.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.7.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.8.json',
 'c:/Users/felip/OneDrive/Documentos/GitHub/dask-examples/data/processed.9.json']

## Convert to Dask Dataframes

Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes.  Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.  

However, Dask Dataframes also expect data that is organized as flat columns.  It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.

In [13]:
b.take(1)

({'age': 25,
  'name': ['Cletus', 'Haney'],
  'occupation': 'Radiographer',
  'telephone': '+18300170997',
  'address': {'address': '407 Cielito Pike', 'city': 'Lafayette'},
  'credit-card': {'number': '4738 1562 8032 9005',
   'expiration-date': '07/18'}},)

In [14]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']   
    }

b.map(flatten).take(1)

({'age': 25,
  'occupation': 'Radiographer',
  'telephone': '+18300170997',
  'credit-card-number': '4738 1562 8032 9005',
  'credit-card-expiration': '07/18',
  'name': 'Cletus Haney',
  'street-address': '407 Cielito Pike',
  'city': 'Lafayette'},)

In [15]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,25,Radiographer,+18300170997,4738 1562 8032 9005,07/18,Cletus Haney,407 Cielito Pike,Lafayette
1,92,Shunter,+1-937-909-2177,4640 7218 0241 3616,02/18,Hank Moran,341 Sunbeam Boulevard,Enid
2,13,Textile Engineer,+1-808-438-3830,3727 562455 30578,03/16,Broderick Case,330 Bocana Boulevard,Leominster
3,47,Advertising Manager,+1-325-806-0524,3400 503900 84972,11/23,Erline Allison,1349 Del Sur Hill,Prichard
4,49,Storewoman,+1-442-995-9947,3477 138247 23951,12/16,Herta Ratliff,907 Farnum Circle,Myrtle Beach


We can now perform the same computation as before, but now using Pandas and Dask dataframe.

In [16]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

2024-07-05 14:19:06,344 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 99c806e2faf8623f7e4157755caf24e8 initialized by task ('shuffle-transfer-99c806e2faf8623f7e4157755caf24e8', 1) executed on worker tcp://127.0.0.1:4282
2024-07-05 14:19:07,724 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 99c806e2faf8623f7e4157755caf24e8 deactivated due to stimulus 'task-finished-1720199947.7225506'


occupation
Tyre Fitter                15
Rig Worker                 15
Roadworker                 15
Amusement Arcade Worker    14
Garda                      14
Polisher                   14
Sheriff Principal          14
Payroll Assistant          13
Seamstress                 13
Lifeguard                  13
Name: count, dtype: int64[pyarrow]

## Learn More

You may be interested in the following links:

-  [Dask Bag Documentation](https://docs.dask.org/en/latest/bag.html)
-  [API Documentation](http://docs.dask.org/en/latest/bag-api.html)
-  [dask tutorial](https://github.com/dask/dask-tutorial), notebook 02, for a more in-depth introduction.